## Data Wrangling

### Introduction

This project is part of a Capstone project for Springboard Data Science Career Track.

The goal of this project is to develop a machine learning model to rank and predict the likelihood that an oil company will initiate a frac job in a county within the Permian Basin in the first quarter of 2024.

In [ ]:
# Import statements
import concurrent.futures as cf
import random
import re
import tempfile
import warnings
from functools import lru_cache
from pathlib import Path
from typing import Optional
from urllib.request import urlopen
from zipfile import ZipFile
import missingno as msno
import cartopy.crs as ccrs
from shapely import wkt
from shapely.geometry import Point

import geopandas as gpd
import geoviews as gv
import geoviews.tile_sources as gts
import colorcet as cc
import holoviews as hv
import hvplot.pandas  # noqa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyproj
from fiona.io import ZipMemoryFile
from pyvis.network import Network
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from tqdm import tqdm

import dask.dataframe as dd
import dask
from dask.diagnostics import ProgressBar
from dask.distributed import Client

hv.extension("bokeh")
gv.extension("bokeh")

In [ ]:
# ignore all warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 40)

In [ ]:
# Test initial print statement
print("CapstoneJourney begins!")

#### Constants
Let's start by defining some constants that will be used throughout this notebook.

Most of the data was first downloaded from external websites and then uploaded onto a cloud storage bucket. This was done to ensure consistency and availability during the project. A brief description of the data and its original source link is referenced below.

## Data Sources

The following table provides an overview of the data sources used in this project:

| Dataset Name | Source URL | Original Source | Description | Date Downloaded |
|--------------|------------|-----------------|-------------|-----------------|
|RegistryUpload Table | [link](https://fracfocus.org/data-download) | FracFocus | This table contains each disclosure’s header information such as the job date, API number, location, base water volume, and total vertical depth. | 2023-11-11 |
RBDMSWells | [link](https://gisdata-occokc.opendata.arcgis.com/datasets/OCCOKC::rbdms-wells/about) | Oklahoma Corporation Commission | This table contains Oklahoma RBDMS statewide well data | 2023-11-23 |
| Wolfcamp Delaware Play Boundary | [link]((https://www.eia.gov/maps/maps.htm))| EIA | Permian Basin, Delaware Sub-Basin: Wolfcamp play boundary (9/4/2018) | 2023-11-19 |
| Wolfcamp Midland Play Boundaries | [link]((https://www.eia.gov/maps/maps.htm))| EIA | Wolfcamp A, B, C, and D play boundaries, Midland Basin (6/4/2020) | 2023-11-21 |
| ShalePlay Delaware | [link]((https://www.eia.gov/maps/maps.htm))| EIA |Delaware play boundary (10/8/2019)  | 2023-11-21 |
| AboYeso GlorietaYeso Spraberry | [link]((https://www.eia.gov/maps/maps.htm))| EIA | Abo-Yeso, Glorieta-Yeso, and Spraberry play boundaries (3/11/2016) | 2023-11-21 |
| NM SLO OilGas Leases | [link](https://www.nmstatelands.org/maps-gis/gis-data-download/)| New Mexico State Land Office | Active Oil and Gas Leases (11/07/2023) | 2023-11-21 |
| NM SLO Geologic Regions | [link]((https://www.nmstatelands.org/maps-gis/gis-data-download/))| New Mexico State Land Office | Geologic Regions (01/04/2010) | 2023-11-21 |
| NM SLO STL Status Combined | [link]((https://www.nmstatelands.org/maps-gis/gis-data-download/))| New Mexico State Land Office | New Mexico State Trust Lands By Subdivision (04/14/2022) | 2023-11-21 |
| All Layers By County | [link](https://rrc.texas.gov/resource-center/research/data-sets-available-for-download/)  | Railroad Commission of Texas | Map & Associated Data: Base Map, Wells, Surveys & Pipelines layers | 2023-11-17 |
| Oil & Gas Leases | [link](https://www.glo.texas.gov/land/land-management/gis/index.html) | Texas General Land Office | Active Leases (11/17/2023) | 2023-11-17 |
| Oil & Gas Units | [link](https://www.glo.texas.gov/land/land-management/gis/index.html) | Texas General Land Office | Active Units (11/17/2023) | 2023-11-17 |
| U.S. County Boundaries | [link](https://www2.census.gov/geo/tiger/TIGER2022/COUNTY/tl_2022_us_county.zip) | United States Census Bureau | County (2022-10-31). Data is downloaded directly in the code. | N/A |
| U.S. County FIPS Codes | [link](https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county) | Wikipedia | List of United States FIPS codes by county. Data is downloaded directly in the code. | N/A |

Each row in the table represents a different dataset. The columns are:

- **Dataset Name**: The name of the dataset.
- **Source URL**: The URL where the dataset can be downloaded. Click on "link" to access the webpage.
- **Original Source**: The original source of the data.
- **Description**: A brief description of the dataset.
- **Date Downloaded**: The date when the dataset was downloaded.

In [ ]:
# Constants
# This cell generates lists of URLs to CSV files stored in a Google Cloud Storage bucket.
# The CSV files contain data from the FracFocus Chemical Disclosure Registry.

# Generate a list of URLs to the FracFocusRegistry CSV files.
# There are 24 files in total, named FracFocusRegistry_i.csv where i ranges from 1 to 24.
DATA_URLS1 = [
    f"https://storage.googleapis.com/mrprime_dataset/fracfocus/FracFocusRegistry_{i}.csv"
    for i in range(1, 25)
]

# Generate a list of URLs to the registryupload CSV files.
# There are 3 files in total, named registryupload_i.csv where i ranges from 1 to 3.
DATA_URLS2 = [
    f"https://storage.googleapis.com/mrprime_dataset/fracfocus/registryupload_{j}.csv"
    for j in range(1, 4)
]

# URL to the readme.txt file in the bucket.
DATA_README_URL = [
    "https://storage.googleapis.com/mrprime_dataset/fracfocus/readme.txt"
]

# url to the OCC (Oklahoma) well data in th bucket
OCC_PARQUET_URL = "https://storage.googleapis.com/mrprime_dataset/capstone_journey/occ/rbdms_wells.parquet"

In [ ]:
# Url for the shapefile for US counties from the Census Bureau's website.
CENSUS_COUNTY_MAP_URL = (
    "https://www2.census.gov/geo/tiger/TIGER2022/COUNTY/tl_2022_us_county.zip"
)
# Url for a Wikipedia page containing a table of FIPS codes for US counties.
FIPS_WIKI_URL = (
    "https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county"
)
# Bounds of the continental US in longitude and latitude.
USA_BOUNDS = (-124.77, 24.52, -66.95, 49.38)
# bounds of the continental US in Web Mercator coordinates.
USA_BOUNDS_MERCATOR = (-13874905.0, 2870341.0, -7453304.0, 6338219.0)

In [ ]:
# url for the shapefiles of Permian Basin, Delaware Sub-Basin: Wolfcamp play boundary
WOLFCAMP_ZIP_URL = "https://storage.googleapis.com/mrprime_dataset/capstone_journey/eia/Wolfcamp_Delaware_Play_Boundary.zip"
MIDLAND_ZIP_URL = "https://storage.googleapis.com/mrprime_dataset/capstone_journey/eia/Wolfcamp_Midland_Play_Boundaries_EIA.zip"
DELAWARE_ZIP_URL = "https://storage.googleapis.com/mrprime_dataset/capstone_journey/eia/ShalePlay_Delaware_EIA.zip"
ABOYESO_ZIP_URL = "https://storage.googleapis.com/mrprime_dataset/capstone_journey/eia/ShalePlays_AboYeso_GlorietaYeso_Spraberry_EIA.zip"
# PB_ZIP_URL = "https://storage.googleapis.com/mrprime_dataset/capstone_journey/eia/PermianBasin_Boundary_Structural_Tectonic.zip"

basins_url_list = [
    WOLFCAMP_ZIP_URL,
    MIDLAND_ZIP_URL,
    DELAWARE_ZIP_URL,
    ABOYESO_ZIP_URL,
    # PB_ZIP_URL,
]


# url for shapefiles of Polygon data set intended to delineate active oil and gas leases on New Mexico State Trust Lands.
NM_SLO_OIL_LEASE_URL = "https://storage.googleapis.com/mrprime_dataset/capstone_journey/nm_slo/OilGas_Leases.zip"

# url for shapefiles of Polygon layer created to highlight general boundaries of subsurface geologic basins and uplifts of New Mexico
NM_SLO_GEO_REGION_URL = "https://storage.googleapis.com/mrprime_dataset/capstone_journey/nm_slo/slo_GeologicRegions.zip"
# url for shapefiles of Polygons of New Mexico State Trust Lands by PLSS subdivision (quarter-quarter, lot, tract, or partial).
NM_SLO_STL_PLSS_URL = "https://storage.googleapis.com/mrprime_dataset/capstone_journey/nm_slo/slo_STLStatusCombined.zip"

nm_slo_url_list = [
    NM_SLO_OIL_LEASE_URL,
    NM_SLO_GEO_REGION_URL,
]  # , NM_SLO_STL_PLSS_URL]

In [ ]:
# Define a list of county numbers that we want to test. These numbers correspond to counties
# that we did not include in the data folder, but they do not cover all 254 counties.

# county numbers are only odd numbers
county_nums = [str(i).zfill(3) for i in range(1, 508) if i % 2]

# Generate a list of URLs to the shapefile zip files stored in a Google Cloud Storage bucket.
# The zip files are named Shp{num}.zip, where {num} is a county number from the county_nums list.
SHP_ZIP_URLS = [
    f"https://storage.googleapis.com/mrprime_dataset/capstone_journey/rrc/all_layers_rrc_20231117/Shp{num}.zip"
    for num in county_nums
]

In [ ]:
# url for the active leases in Texas on State land gdb
GDB_ZIP_URLS = [
    "https://storage.googleapis.com/mrprime_dataset/capstone_journey/glo/GDB_ActiveLeases.zip",
    "https://storage.googleapis.com/mrprime_dataset/capstone_journey/glo/GDB_ActiveUnits.zip",
    # "https://storage.googleapis.com/mrprime_dataset/capstone_journey/glo/GDB_InactiveLeases.zip",
]

#### Function definations
Next, let's define some functions that will be used throughout this notebook.

In [ ]:
def read_csv_concurrent(urls_list):
    """Reads a list of CSV files concurrently"""
    # Create a thread pool
    with cf.ThreadPoolExecutor() as executor:
        # Use map to apply pd.read_csv to each URL
        results = list(tqdm(executor.map(pd.read_csv, urls_list), total=len(urls_list)))
    # Return the results
    return results

In [ ]:
def extract_specific_gdf_from_local_zip(
    zip_paths: list[str], regex_patterns: list[str]
) -> dict[str, gpd.GeoDataFrame]:
    """
    Reads shapefiles from a list of zip files and returns a dictionary
    where the keys are the names of the shapefiles and the values are GeoDataFrames.
    """
    # Initialize an empty dictionary to store the GeoDataFrames
    shp_dict = {}
    # compile the regex patterns
    patterns = [re.compile(pattern) for pattern in regex_patterns]

    # Loop over the list of zip file paths
    for zip_path in zip_paths:
        # Open the zip file
        with ZipFile(zip_path) as z:
            # Get the list of files in the zip file
            zip_contents = z.namelist()
            # Filter the list to get only the shapefiles that match any of the patterns
            shp_files = [
                f
                for f in zip_contents
                for pattern in patterns
                if pattern.search(f) and f.endswith(".shp")
            ]
            # read the shapefiles into GeoDataFrames
            for shp_file in shp_files:
                # Get the name of the shapefile
                shp_name = Path(shp_file).stem
                # Read the shapefile into a GeoDataFrame and add it to the dictionary
                shp_dict[shp_name] = gpd.read_file(f"zip://{zip_path}!{shp_file}")
    # Return the dictionary of GeoDataFrames
    return shp_dict

In [ ]:
def extract_matching_shp_files_from_zip_urls(
    zip_urls: list[str], regex_patterns: list[str]
) -> dict[str, gpd.GeoDataFrame]:
    """
    Reads shapefiles from a list of zip file urls and returns a dictionary
    where the keys are the names of the shapefiles and the values are GeoDataFrames.
    """
    # Initialize an empty dictionary to store the GeoDataFrames
    shp_dict = {}
    # compile the regex patterns
    patterns = [re.compile(pattern) for pattern in regex_patterns]

    # Loop over the list of zip file urls
    for zip_url in tqdm(zip_urls, desc="Processing zip files"):
        # download the zip file
        with urlopen(zip_url) as u:
            zip_data = u.read()
        # create a ZipMemoryFile from the zip data
        with ZipMemoryFile(zip_data) as z:
            # get the list of files in the zip file
            zip_files = z.listdir()
            # filter for shapefiles that match any of the patterns
            shp_files = [
                f
                for f in zip_files
                for pattern in patterns
                if pattern.search(f) and f.endswith(".shp")
            ]
            # read the shapefiles into GeoDataFrames
            for shp_file in shp_files:
                with z.open(shp_file) as f:
                    shp_dict[Path(shp_file).stem] = gpd.GeoDataFrame.from_features(
                        f, crs=f.crs
                    )
    # Return the dictionary of GeoDataFrames
    return shp_dict

In [ ]:
def process_zip_url(
    zip_url: str, patterns: list[re.Pattern]
) -> dict[str, gpd.GeoDataFrame]:
    """Downloads a zip file url and returns a dictionary of GeoDataFrames for shapefiles that match the patterns"""
    shp_dict = {}
    with urlopen(zip_url) as u:
        zip_data = u.read()
    with ZipMemoryFile(zip_data) as z:
        zip_files = z.listdir()
        shp_files = [
            f
            for f in zip_files
            for pattern in patterns
            if pattern.search(f) and f.endswith(".shp")
        ]
        for shp_file in shp_files:
            with z.open(shp_file) as f:
                shp_dict[Path(shp_file).stem] = gpd.GeoDataFrame.from_features(
                    f, crs=f.crs
                )
    return shp_dict


def extract_matching_shp_files_from_zip_urls_concurrent(
    zip_urls: list[str], regex_patterns: list[str]
) -> dict[str, gpd.GeoDataFrame]:
    """Reads shapefiles from a list of zip file urls and returns a dictionary
    where the keys are the names of the shapefiles and the values are GeoDataFrames."""
    shp_dict = {}
    patterns = [re.compile(pattern) for pattern in regex_patterns]
    with cf.ThreadPoolExecutor() as executor:
        future_to_url = {
            executor.submit(process_zip_url, url, patterns): url for url in zip_urls
        }
        futures = tqdm(
            cf.as_completed(future_to_url),
            total=len(future_to_url),
            desc="Processing URLs",
            dynamic_ncols=True,
        )
        for future in futures:
            shp_dict.update(future.result())
    return shp_dict

In [ ]:
def concat_gdf_from_dict(gdf_dict: dict[str, gpd.GeoDataFrame]) -> gpd.GeoDataFrame:
    """
    Given a dictionary of GeoDataFrames, returns a single GeoDataFrame
    with a new column indicating the source of the data.
    """
    # use a dictionary comprehension to create a new dictionary
    gdf_data = {k: gdf.assign(source_file=k) for k, gdf in gdf_dict.items()}
    # return the concatenated GeoDataFrame
    return pd.concat(gdf_data.values(), ignore_index=True)

In [ ]:
def extract_gdfs_from_zip(zip_path: str) -> Optional[dict[str, gpd.GeoDataFrame]]:
    """
    Reads shapefiles from a zip file and returns a dictionary of GeoDataFrames.
    """
    gdfs = {}
    # Open the zip file
    with ZipFile(zip_path) as z:
        # Get the list of files in the zip file
        zip_contents = z.namelist()
        # Find the shapefiles
        shp_files = [f for f in zip_contents if f.endswith(".shp")]
        for shp_file in shp_files:
            # Read the shapefile into a GeoDataFrame
            gdf = gpd.read_file(f"zip://{zip_path}!{shp_file}")
            gdfs[shp_file] = gdf

    # If no shapefile was found, return None
    return gdfs if gdfs else None

In [ ]:
def extract_gdfs_from_zip_url(zip_url: str) -> Optional[dict[str, gpd.GeoDataFrame]]:
    """
    Downloads a ZIP file from a URL, reads shapefiles from the ZIP file, and returns a dictionary of GeoDataFrames.
    """
    gdfs = {}
    # Open the URL
    with urlopen(zip_url) as u:
        # Read the content of the response into a byte stream
        zip_data = u.read()
        # Open the ZIP file from the byte stream
        with ZipMemoryFile(zip_data) as z:
            # Get the list of files in the ZIP file
            zip_contents = z.listdir()
            # Find the shapefiles
            shp_files = [f for f in zip_contents if f.endswith(".shp")]
            for shp_file in shp_files:
                # Read the shapefile into a GeoDataFrame
                with z.open(shp_file) as f:
                    gdf = gpd.GeoDataFrame.from_features(f, crs=f.crs)
                gdfs[Path(shp_file).stem] = gdf

    # If no shapefile was found, return None
    return gdfs if gdfs else None

In [ ]:
def process_shp_url(zip_url: str):
    """Downloads a zip file url and returns a dictionary of GeoDataFrames for shapefiles that match the patterns"""
    shp_dict = {}
    with urlopen(zip_url) as u:
        zip_data = u.read()
    with ZipMemoryFile(zip_data) as z:
        zip_files = z.listdir()
        shp_files = [f for f in zip_files if f.endswith(".shp")]
        for shp_file in shp_files:
            with z.open(shp_file) as f:
                shp_dict[Path(shp_file).stem] = gpd.GeoDataFrame.from_features(
                    f, crs=f.crs
                )
    return shp_dict


def extract_gdfs_from_zip_url_concurrent(
    zip_urls: list[str],
) -> dict[str, gpd.GeoDataFrame]:
    """Reads shapefiles from a list of zip file urls and returns a dictionary
    where the keys are the names of the shapefiles and the values are GeoDataFrames."""
    shp_dict = {}
    with cf.ThreadPoolExecutor() as executor:
        future_to_url = {executor.submit(process_shp_url, url): url for url in zip_urls}
        futures = tqdm(
            cf.as_completed(future_to_url),
            total=len(future_to_url),
            desc="Processing URLs",
            dynamic_ncols=True,
        )
        for future in futures:
            shp_dict.update(future.result())
    return shp_dict

In [ ]:
def read_gdb_from_zip(gdb_zips_list: list[str]):
    """Reads a list of zip files containing geodatabases and returns a dictionary of GeoDataFrames"""
    # initialize an empty dictionary
    gdb_dict = {}
    # loop through each zip file
    for gdb_zip in gdb_zips_list:
        with ZipFile(gdb_zip, "r") as z:
            # get list of files in zip
            files = z.namelist()
            # filter for gdb folders
            gdb_folders = [f for f in files if f.endswith(".gdb/")]
            # if there is a gdb folder in the zip file
            if gdb_folders:
                # get it and read it into a GeoDataFrame
                gdb_folder = gdb_folders[0]
                gdb_dict[Path(gdb_folder).stem] = gpd.read_file(
                    f"zip://{gdb_zip}!{gdb_folder}"
                ).to_crs("EPSG:4269")
    # return the dictionary of GeoDataFrames
    return gdb_dict

In [ ]:
def read_gdb_from_zip_url(gdb_urls_list: list[str]):
    """Reads a list of zip file urls containing geodatabases and returns a dictionary of GeoDataFrames"""
    # initialize an empty dictionary
    gdb_dict = {}
    # loop through each zip file
    for gdb_url in gdb_urls_list:
        # create a temporary directory
        with tempfile.TemporaryDirectory() as tmp_dir:
            # download the zip file
            with urlopen(gdb_url) as u, open(f"{tmp_dir}/data.zip", "wb") as f_out:
                f_out.write(u.read())
            # extract the zip file
            with ZipFile(f"{tmp_dir}/data.zip", "r") as zip_ref:
                zip_ref.extractall(tmp_dir)
            # get the list of extracted files
            extracted_files = list(Path(tmp_dir).iterdir())
            # filter for gdb folders
            gdb_folders = [f for f in extracted_files if f.suffix == ".gdb"]
            # if there is a gdb folder in the extracted files
            if gdb_folders:
                # get it and read it into a GeoDataFrame
                gdb_folder = gdb_folders[0]
                gdb_dict[Path(gdb_folder).stem] = gpd.read_file(gdb_folder).to_crs(
                    "EPSG:4269"
                )
    # return the dictionary of GeoDataFrames
    return gdb_dict

In [ ]:
def process_gdb_url(gdb_url):
    """Downloads a zip file url containing a geodatabase and returns a GeoDataFrame"""
    with tempfile.TemporaryDirectory() as tmp_dir:
        # download the zip file
        with urlopen(gdb_url) as u, open(f"{tmp_dir}/data.zip", "wb") as f_out:
            f_out.write(u.read())
        # extract the zip file
        with ZipFile(f"{tmp_dir}/data.zip", "r") as zip_ref:
            zip_ref.extractall(tmp_dir)
        # get the list of extracted files
        extracted_files = list(Path(tmp_dir).iterdir())
        # filter for gdb folders
        gdb_folders = [f for f in extracted_files if f.suffix == ".gdb"]
        # if there is a gdb folder in the extracted files
        if gdb_folders:
            # get it and read it into a GeoDataFrame
            gdb_folder = gdb_folders[0]
            return Path(gdb_folder).stem, gpd.read_file(gdb_folder)


def read_gdb_from_zip_url_concurrent(gdb_urls_list: list[str]):
    """Reads a list of zip file urls containing geodatabases and returns a dictionary of GeoDataFrames"""
    # initialize an empty dictionary
    gdb_dict = {}
    # create a ThreadPoolExecutor
    with cf.ThreadPoolExecutor() as executor:
        # submit the process_gdb_url function for each url and gather the results
        future_to_url = {
            executor.submit(process_gdb_url, url): url for url in gdb_urls_list
        }
        for future in cf.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                key, data = future.result()
                gdb_dict[key] = data
            except Exception as exc:
                print(f"{url} generated an exception: {exc}")
    # return the dictionary of GeoDataFrames
    return gdb_dict

In [ ]:
# Function definitions
def pascal_to_snake(name: str):
    """Converts a string from PascalCase to snake_case"""
    # (?<=[A-Za-z0-9]) - positive lookbehind for any alphanumeric character
    # (?=[A-Z][a-z]) - positive lookahead for any uppercase followed by lowercase
    pattern = re.compile(r"(?<=[A-Za-z0-9])(?=[A-Z][a-z])")
    name = pattern.sub("_", name).lower()
    return name

In [ ]:
def unify_crs(
    dataframe: pd.DataFrame,
    lon_col: str = "longitude",
    lat_col: str = "latitude",
    crs_col: str = "crs",
    final_crs: str = "EPSG:4269",
):
    """
    Given a DataFrame with lon/lat or x/y coordinates,
    converts the coordinates to a unified crs and combines
    into a single GeoDataframe with a geometry column.
    """

    # Define the main columns that will be used for the conversion
    main_cols = [lon_col, lat_col, crs_col]

    # Get the other columns in the dataframe
    other_cols = list(set(dataframe.columns) - set(main_cols))

    # Create a subframe with only the main columns
    subframe = dataframe[main_cols]

    # Create a list of GeoDataFrames, each with a different CRS
    geo_dfs = [
        gpd.GeoDataFrame(
            # Use the data for this CRS
            data=data,
            # Create a geometry column from the lon/lat columns
            geometry=gpd.points_from_xy(x=data[lon_col].values, y=data[lat_col].values),
            # Set the CRS for this GeoDataFrame
            crs=pyproj.CRS(crs_val),
            # Convert the GeoDataFrame to the final CRS
        ).to_crs(final_crs)
        # Do this for each unique CRS in the subframe
        for crs_val, data in subframe.groupby(crs_col)
    ]

    # Merge the GeoDataFrames back together and return the result
    return pd.merge(
        # Concatenate the GeoDataFrames
        pd.concat(geo_dfs, sort=True),
        # Add the other columns back in
        dataframe[other_cols],
        # Merge on the index
        left_index=True,
        right_index=True,
    )

In [ ]:
# @lru_cache(maxsize=3)
def get_background_map(bgcolor="black", alpha=0.5):
    """Returns a GeoViews background map"""
    return gts.CartoLight().opts(bgcolor=bgcolor, alpha=alpha)


def platecaree_to_mercator_vectorised(x, y):
    """Use Cartopy to convert PlateCarree coordinates to Mercator"""
    return ccrs.GOOGLE_MERCATOR.transform_points(ccrs.PlateCarree(), x, y)[:, :2]

In [ ]:
def format_in_000(num):
    """Formats a number in thousands"""
    for unit in ["", "thousand", "million", "billion", "trillion"]:
        if abs(num) < 1000.0:
            return f"{num:3.2f} {unit}"
        num /= 1000.0
    return f"{num:.2f} quadrillion"

In [ ]:
def split_datetime(df, column):
    """Splits a datetime column into year, month, and day columns"""
    # remove '_date' from the column name
    column_stem = column.replace("_date", "") if "_date" in column else column
    try:
        datetime_series = pd.to_datetime(df[column], errors="coerce")
        if datetime_series.isna().any():
            print(f"Errors occurred during conversion of column {column}.")
        df[column_stem + "_year"] = datetime_series.dt.year
        df[column_stem + "_month"] = datetime_series.dt.month
        df[column_stem + "_day"] = datetime_series.dt.day
    except KeyError:
        print(f"Column {column} not found in the DataFrame.")
    except Exception as e:
        print(f"An error occurred: {e}")

### Load data

First dataset is from FracFocus. There is also a readme file which contains the data dictionary for the dataset. Let's have a look at both.

Readme file with data dictionary

In [ ]:
# get readme data
readme = urlopen(DATA_README_URL[0]).read().decode("windows-1252")
display(readme)

In [ ]:
# print function goes beyond 'hello world' and takes care of the escape characters
print(readme)

In [ ]:
# We can collect all the dataframe into a list and then concatenate them
df_list = read_csv_concurrent(DATA_URLS2)


dfs = pd.concat(df_list).reset_index(drop=True)

In [ ]:
registry_df = pd.DataFrame()
registry_df = dfs.copy()
registry_df.info()

Looking at the missing values it is interesting to see that most missing values are from the `TVD`, `TotalBaseWaterVolume` and `TotalBaseNonWaterVolume`. One reason for this may be found in the data limitations on terms of use on the FracFocus website. It states:
-  Disclosures submitted using the FracFocus 1.0 format (January, 2011 to May 31, 2013) will contain only header data. 
-  Disclosures submitted using the FracFocus 2.0 format (November 2012 to present) will contain both header and chemical data. NOTE: Between November, 2012 and May 31, 2013 disclosures in both 1.0 and 2.0 formats were submitted to the system. 
-  After May 31, 2013 only disclosures submitted in the 2.0 format were accepted.
-  Data submitted appears as it was submitted by the operator or operator’s authorized agent. FracFocus does not warrant the data in any way.

In [ ]:
# Calculate the percentage of non-missing values in each column
missing_data_percent = (registry_df.notna().mean() * 100).rename("Percent")

# Create a DataFrame of the counts of non-missing values
non_missing_count = registry_df.notna().sum().rename("Count")

# Concatenate the two DataFrames along the columns
non_missing_data = pd.concat([missing_data_percent, non_missing_count], axis=1)

# Create a horizontal bar plot of the percentage of non-missing data
hbar_plot = non_missing_data.hvplot.barh(
    y="Percent",
    width=800,
    height=600,
    title="Percentage of Non-Missing Data in Each Column",
    ylabel="",
    xlabel="",
    xaxis="bare",
    hover_cols="all",
).opts(
    active_tools=["box_zoom"],
    toolbar="above",
)

hbar_plot

In [ ]:
# Look at some of the rows of the dataframe
display(registry_df.head(3))
display(registry_df.sample(5, random_state=628))
display(registry_df.tail(3))

From our first look at a few sample rows some things stick out immediately.
1. The dataset may be in chronological order and the values of the `JobStartDate`/`JobEndDate` at both of the extremes may be incorrect.
2. There may be an abundance for `StateNumber` `42` if 4 out of the 5 draws of the 200k+ rows drawn at random had a `StateNumber` of `42`.



### Data Cleaning

Before we jump into cleaning the data in the columns, let's make the columns look more pythonic by changing the column names to snake_case.


In [ ]:
registry_df.columns = [pascal_to_snake(col) for col in registry_df.columns]
registry_df.info(memory_usage="deep")

Next, we can remove the columns with only null values. These are the last 2 columns in the dataframe, `source` and `dtmod`. Also we can drop the `total_non_base_water_volume` column since we may not have much need for it.


In [ ]:
registry_df = registry_df.drop(
    columns=["source", "dtmod", "total_base_non_water_volume"]
)
registry_df.info(memory_usage="deep")

Next, we will fix some of the dtypes of the columns.
- Both the `job_start_date` and the `job_end_date` columns are object dtypes, so we will convert those to datetime dtypes and drop the timestamp.
- We can also separate out the date components into its various components. This may come in handy for feature engineering later on.
- The `projection` column is an object dtype. That can be converted to a string dtype and shorten to `crs` as it represents the Cooordinate Reference System used in the `latitude` and `longitude` columns values. We can dig into what CRS is later on.
- The `federal_well` and `indian_well` columns are both boolean type columns. They may be more aptly named as `is_federal_well` and `is_indian_well` respectively.

In [ ]:
# Use the function on 'job_start_date' and 'job_end_date'
split_datetime(registry_df, "job_start_date")
split_datetime(registry_df, "job_end_date")
registry_df[[col for col in registry_df.columns if re.search("start|end", col)]].info(
    memory_usage="deep"
)
# show the values which are null still
registry_df[
    registry_df[[col for col in registry_df.columns if re.search("start|end", col)]]
    .isna()
    .any(axis=1)
].shape

In [ ]:
# Convert 'job_start_date' to datetime format and format it as 'YYYY-MM-DD'
registry_df["job_start_date"] = pd.to_datetime(
    registry_df["job_start_date"], errors="coerce"
).dt.strftime("%Y-%m-%d")

# Convert 'job_end_date' to datetime format and format it as 'YYYY-MM-DD'
registry_df["job_end_date"] = pd.to_datetime(
    registry_df["job_end_date"], errors="coerce"
).dt.strftime("%Y-%m-%d")

# drop rows with null values in 'job_start_date' and 'job_end_date'
# registry_df = registry_df.dropna(subset=["job_start_date", "job_end_date"])


# Rename some columns for clarity
registry_df.rename(
    columns={
        "federal_well": "is_federal_well",
        "indian_well": "is_indian_well",
        "projection": "crs",
    },
    inplace=True,
)

# Display the information of the DataFrame
registry_df.info(memory_usage="deep")

Next, we will look at the `api_number` column.
We learned from the read me that 
> APINumber - The American Petroleum Institute well identification number formatted as follows xx-xxx-xxxxx0000 Where: 
> - First two digits represent the state, 
> - second three digits represent the county, 
> - third 5 digits represent the well.

Theoretically, we could just grab the first two characters of the `APINnumber` and use that as the state number according to the definition of the `APINumber` above. Actually, that would not be a good idea, and here is why.<br>
Although the column was called `APINumber`, it is not actually a number, so if it starts with a leading `0` that first character `0`, cannot be omitted from the value. Let's look at some of the rows with a single digit state numbers.

Right now, 
- the `api_number` column is an object dtype, but a better option would be a `string` dtype, as `object` dtype can be mixed . We can also shorten that column name to `api`.
- the `state_number` column and the `county_number` column are both `int64` dtypes right now. `string` type may be a stronger option.
- `state_code` and `county_code` may be better names for the `state_number` and `county_number` columns respectively.


In [ ]:
# rows where the state_number is a single digit
registry_df[
    (registry_df["state_number"] == 3) | (registry_df["state_number"] == 5)
].sample(5, random_state=628)

Some rows' `api_number` values have leading `0`, which is correct, but some do not. The rows without the leading `0` though are 13 characters long instead of 14. Maybe we can just add a leading `0` where needed until all API number values are 14 characters long. 

In [ ]:
# Check the number of characters in the api_number column
registry_df["api_number"].astype("string").str.len().value_counts()

Most are 14 characters long, but some are 13 characters long, like the ones we saw above without the leading `0`. Let's assume the ones with 13 characters are missing the leading `0` and not something else.

In [ ]:
# Convert 'api' to string and pad it with zeros to make it 14 characters long
registry_df["api"] = registry_df["api_number"].astype("string").str.zfill(14)

# Convert 'state_number' to string and pad it with zeros to make it 2 characters long
registry_df["state_code"] = registry_df["state_number"].astype("string").str.zfill(2)

# Convert 'county_number' to string and pad it with zeros to make it 3 characters long
registry_df["county_code"] = registry_df["county_number"].astype("string").str.zfill(3)

In [ ]:
# check which rows may have the api with the first two digits not matching the state number
api_state_mismatch_mask = registry_df["state_code"] != registry_df["api"].str[0:2]
# api_state_mismatch_mask

In [ ]:
# check which rows may have the api with the first two digits not matching the state number
registry_df[api_state_mismatch_mask][
    ["api_number", "api", "state_code", "state_name", "county_code", "county_name"]
]

We expected to get 2 rows here, since we checked the length of the `api_number` column above we saw that 1 row had 10 and another row had 12 characters. It is only two rows, so this may be an easy fix.

In [ ]:
# Remove leading zeros and pad to 14 digits on mismatches
registry_df.loc[api_state_mismatch_mask, "api"] = (
    registry_df.loc[api_state_mismatch_mask, "api"].str.lstrip("0").str.ljust(14, "0")
)

In [ ]:
# check which rows may have the api with the first two digits not matching the state number
registry_df[api_state_mismatch_mask][
    ["api_number", "api", "state_code", "state_name", "county_code", "county_name"]
]

In [ ]:
# check which rows may have the api with the 3-5 digits not matching the county number
api_county_mismatch_mask = registry_df["county_code"] != registry_df["api"].str[2:5]
registry_df[api_county_mismatch_mask]

State name should not have more than 50 possible values, given that there are only 50 states in the US. If we were to check the number of unique values in the `state_name` column, we would see 95. This is due to the variation in the way the `state_name` value is entered. Although not as obvious, we can assume the same for the `county_name` column. Luckily, the `api` includes both the `state_number` and the `county_number`. With this we can do 
1. data validation ensuring that these corresponding columns match
2. Ensure that the `state_name` and the `county_name` columns are correct. Important to note that 
> The state codes used in an API number are DIFFERENT from another standard which is the Federal Information Processing Standard (FIPS) state code established in 1987 by NIST. ([source](https://en.wikipedia.org/wiki/API_well_number#State_code))

In [ ]:
print(
    f'Number of different values in state_name column: {registry_df["state_name"].nunique()}'
)
print(
    f'Number of different values in state_number column: {registry_df["state_number"].nunique()}'
)

In [ ]:
# group by state_code and find the mode of the state_name
state_code_mode = (
    registry_df.groupby("state_code")["state_name"]
    .apply(lambda x: x.mode().iloc[0])
    .reset_index()
)
state_code_mode

In [ ]:
registry_df = registry_df.merge(state_code_mode.rename(columns={"state_name": "state"}))
registry_df.sample(3, random_state=628)

We will focus our efforts in the most recent 10 years. Although more data is usually better, data too far in the past may distract whatever model we may build since unconventional drilling practices have really taken over the industry. We will also put our focus in one specific area, the Permian Basin. The Permian Basin has been instrumental in the shale boom transformation and is the most active area of exploration and production in the US presently. 

In [ ]:
# create mask for from 2013 onwards
post_2012_mask = registry_df["job_start_date"] >= "2013-01-01"
registry_df_post_2012 = registry_df[post_2012_mask].copy()

# find all the rows with null values
null_mask = registry_df_post_2012.isna().any(axis=1)
registry_df_post_2012[null_mask]

See how many nans we still have in each column.

In [ ]:
registry_df_post_2012.info(memory_usage="deep")
registry_df_post_2012.isna().sum()

Looking at the `county_number` for the rows with a nan value in the `county_name` column, we can see why there is a nan for the `county_name`. Those numbers are most likely incorrect as small states like `North Dakota` and `Arkansas` do not have large `county_number` values. However we can still try to impute what the correct values by cross referencing with other sources or by using the `latitude` and `longitude` values.


In [ ]:
# the rows with a null value for the county_name column
registry_df_post_2012[registry_df_post_2012["county_name"].isna()]

In [ ]:
# get the index of one of the rows with a null value for the county_name column (3rd one down)
index_vanorsdale = registry_df_post_2012.query("api_number == '03729439000000'").index

#### Oklahoma Commission Corporation (OOC)

With some search engine investigating, we can learn that WFD Oil Corporation is a PC in Oklahoma. We also learn that the well name is `VANORSDOL` ,the well number is `#1-29`, and the API number is `3503729439` `0000`. We can correct some of the data which was entered incorrectly in FracFocus.

The data are looking for is in this markdown cell so we can manually input it in but we will do it through code instead. We will query the api number in the data we have on the wells in OK.

| Column Name | Value |
| --- | --- |
API	|3503729439
WELL_NAME|	VANORSDOL
WELL_NUM|	#1-29
OPERATOR|	WFD OIL CORPORATION
WELLSTATUS|	AC
WELLTYPE|	OIL
SH_LAT	|35.749381
SH_LON	|-96.370355
COUNTY	|CREEK


In [ ]:
# When reading the Parquet file
occ_wells = pd.read_parquet(OCC_PARQUET_URL)
# Convert the WKT column back to a geometry column
occ_wells["geometry"] = occ_wells["geometry"].apply(lambda x: wkt.loads(x))


# Convert the DataFrame to a GeoDataFrame, specifying the CRS
occ_wells = gpd.GeoDataFrame(
    occ_wells, geometry="geometry", crs=occ_wells["crs"].iloc[0]
)
occ_wells.info()
# look at 1 sample row of the dataframe
occ_wells.sample()

In [ ]:
# Convert the 'api' column to string
occ_wells["api"] = occ_wells["api"].astype("int64").astype("string")
occ_wells.sample()

In [ ]:
# make a copy of the well_name column
registry_df_post_2012["well"] = registry_df_post_2012["well_name"].copy()

In [ ]:
registry_df_post_2012[registry_df_post_2012["api"].str.contains("11533124")]

In [ ]:
# query the well_name column for 'vanors
# occ_wells[occ_wells["well_name"].str.contains("vanors", case=False, na=False)]
vanorsdol_row = occ_wells.query(
    'well_name.fillna("").str.contains("vanors", case=False) & (api == "3503729439")',
)[
    [
        "api",
        "well_name",
        "well_num",
        "operator",
        "sh_lat",
        "sh_lon",
        "county",
    ]
].rename(
    columns={"sh_lat": "latitude", "sh_lon": "longitude"}
)
vanorsdol_row["well"] = (
    vanorsdol_row["well_name"].str.title()
    + " "
    + vanorsdol_row["well_num"].astype("string")
)
index_vanorsdol = vanorsdol_row.index

columns_to_replace = ["api", "well", "latitude", "longitude"]
for col in columns_to_replace:
    registry_df_post_2012.loc[index_vanorsdale, col] = vanorsdol_row.loc[
        index_vanorsdol, col
    ].values

# check that the values have been replaced
registry_df_post_2012.loc[index_vanorsdale, columns_to_replace]

In [ ]:
# adjust the api column to 14 characters again
registry_df_post_2012["api"] = registry_df_post_2012["api"].str.ljust(14, "0")

registry_df_post_2012[registry_df_post_2012["county_name"].isna()]

Now, with `latitude` and `longitude` coordinates for all 4 rows with missing `county_name`, let's find out which counties they belong to spatially.



### Geodataframe

We can get the boundary coordinates for all the counties in the US from the [census.gov](https://www.census.gov/) website. We saved the URL for this as `CENSUS_COUNTY_MAP_URL` at the top of the notebook.

In [ ]:
# read in the US counties map data using geopandas
county = gpd.read_file(CENSUS_COUNTY_MAP_URL)[
    ["GEOID", "STATEFP", "COUNTYFP", "NAME", "geometry"]
]
county.columns = county.columns.str.lower()
county.sample(3)

We will scrape the FIPS table from wikipedia since the county dataframe does not have the state name and merge the 2 tables just for convenience. 

In [ ]:
fips_df = pd.read_html(FIPS_WIKI_URL)[1]
fips_df.columns = ["geoid", "county", "state"]
fips_df["geoid"] = fips_df["geoid"].astype("string").str.zfill(5)
fips_df.sample(3)

In [ ]:
county_fips_gdf = county.merge(fips_df, on="geoid")
county_fips_gdf.sample(3, random_state=628)

Quick note about `GeoDataFrames`: they must have a column called `geometry` and this column contains the geometric objects. This call is what enables `geopandas` to perform spatial operations, and can also contain certain attributes like `.crs` which is the coordinate reference system.


Commonly used datums in North America are NAD27, NAD83, and WGS84. More info [here](https://webhelp.esri.com/arcgisdesktop/9.3/index.cfm?TopicName=Projection_basics_the_GIS_professional_needs_to_know).<br>

The county geodataframe uses `EPSG:4269` which is the EPSG code for the NAD83 coordinate system. Let's create a geodataframe with the `latitude` and `longitude` values that we have and put all of the points to the same CRS.

In [ ]:
county_fips_gdf.crs

In [ ]:
# ensures each row of the geodataframe is in the same CRS
registry_gdf = unify_crs(registry_df_post_2012, crs_col="crs")

In [ ]:
# We can now perform a spatial join on the 2 GeoDataFrame.
# fitler for rows with null county_name

joined_gdf = (
    registry_gdf[registry_gdf["county_name"].isna()]
    .sjoin(county_fips_gdf.drop(columns=["county"]), how="left", predicate="intersects")
    .drop(columns=["index_right"])
)
joined_gdf

- The `North Dakota` `county_number` should be `061`, which is `Mountrail` county, not `610`. 
- The `Utah` `county_number` though was actually correct. The error was the state number which should have been `42`, not `43`. This error is somewhat significant as according to the data dictionary:
> APINumber - The American Petroleum Institute well identification number formatted as follows xx-xxx-xxxxx0000 Where: First two digits 
represent the state, second three digits represent the county, third 5 digits represent the well.<br>

All this means is the `api` number is also incorrect. It should be `42317428660000` (<u><b>42</b></u>-317-42866-0000) instead of `43317428660000` (<u><b>43</b></u>-317-42866-0000).<br>


In [ ]:
# Let's corect theos values putting the county_code and
registry_gdf["county"] = registry_gdf["county_name"].copy()

# replace the county_code and county columns with the values from the joined_gdf
registry_gdf.loc[joined_gdf.index, "county"] = joined_gdf["name"]
registry_gdf.loc[joined_gdf.index, "county_code"] = joined_gdf["countyfp"]

# change the api column of the last row in joined_gdf to 42317428660000 instead of 43317428660000
# registry_gdf.loc[joined_gdf.index[-1], "api"] = "42317428660000"
registry_gdf["api"] = registry_gdf["api"].replace("43317428660000", "42317428660000")
# correct the state_code values for where the api was changed
registry_gdf["state_code"] = registry_gdf["api"].str[0:2]
# Create a mapping from 'state_code' to 'state'
state_mapping = state_code_mode.set_index("state_code")["state"].to_dict()

# Use the mapping to update the 'state' column in 'registry_gdf'
registry_gdf["state"] = registry_gdf["state_code"].map(state_mapping)


# check that the values have been replaced
registry_gdf[registry_gdf["county_name"].isna()]

In [ ]:
registry_gdf.columns

Another way to impute the missing `county_name` values would have been by using the `well_name` in the `registry_df_post_2012` dataframe. Assuming the other wells on the same pad has the correct `state_code` and `state` values. However, this may not have worked with `Vanorsdol` as there's only one well with that name.

In [ ]:
registry_df_post_2012[
    registry_df_post_2012["well_name"].str.contains("vanors", case=False, na=False)
]

In [ ]:
# show other wells with a similar name to rhea 1-6
registry_df_post_2012[
    registry_df_post_2012["well"].str.contains("Rhea 1-6", case=False, na=False)
]

In [ ]:
registry_df_post_2012[registry_df_post_2012["well"].str.contains("Trulson", case=False)]

In [ ]:
# county_fips_gdf[county_fips_gdf["state"].isin(permian_states)]
registry_df_post_2012[registry_df_post_2012["well"].str.contains("palermo", case=False)]

In [ ]:
# make the well column uppercase to lower variation among for wells on the same pad
registry_gdf["well"] = registry_gdf["well"].str.upper()
# make the operator column uppercase to lower the variation among entry for the same operator
registry_gdf["operator"] = registry_gdf["operator_name"].str.upper()

In [ ]:
registry_gdf.columns

Just as we did for those 4 wells to find the `county` and `state` for which they belong to, we can do that for all the wells.

In [ ]:
# create 2 new columns, lat and lon in the registry_gdf for corrections
registry_gdf["lat"] = registry_gdf["latitude"].copy()
registry_gdf["lon"] = registry_gdf["longitude"].copy()

# Check which other wells may have a state value which did not agree with the spatial join result
registry_county_gdf = registry_gdf.sjoin(
    county_fips_gdf, how="left", predicate="intersects"
).drop(columns=["index_right"])

trimmed_column_set = [
    "api",
    "well",
    "state_left",
    "state_right",
    "county_left",
    "county_right",
    "county_code",
    "operator",
    "lon",
    "lat",
    "geometry",
]

# rows whichthe spatial join did not match for the both dataframes
# mismatch_geo = registry_county_gdf[registry_county_gdf["state_right"].isna()]
mismatch_geo = registry_county_gdf[
    registry_county_gdf["state_left"] != registry_county_gdf["state_right"]
]
print(f"Number of rows: {len(mismatch_geo)}")

The ones with a nan in the `state_right` column are those that we could not find a match for based on the `geometry`. We can match those for OK using the `api` number and see if the coordinates match for the wells in FracFocus match those from the OCC.

In [ ]:
# get the rows with Oklahoma in the state_left column
mismatch_geo_ok = mismatch_geo.query('state_left.str.contains("Oklahoma")')
print(f"Number of rows from OK state: {len(mismatch_geo_ok)}")
mismatch_geo_ok[trimmed_column_set].sample(3)

In [ ]:
# Store the original index in a new column
mismatch_geo_ok["original_index"] = mismatch_geo_ok.index
# alter api to match the format in the occ_wells dataframe
mismatch_geo_ok["ok_api"] = mismatch_geo_ok["api"].str[:10]  # ok for Oklahoma
# drop the api column
mismatch_geo_ok.drop(columns=["api"], inplace=True)
# rename the api column in occ_wells to ok_api
occ_wells.rename(columns={"api": "ok_api"}, inplace=True)
# look for the api in the occ_wells dataframe and merge that row to mismatch_geo_ok
joined_mismatch_ok = mismatch_geo_ok.merge(
    occ_wells[
        ["ok_api", "well_name", "well_num", "operator", "sh_lat", "sh_lon", "county"]
    ],
    how="left",
    on="ok_api",
)
joined_mismatch_ok[["geometry", "latitude", "longitude", "sh_lat", "sh_lon"]]

In [ ]:
# replace the lat and lon values with the sh_lat and sh_lon values
# Replace the 'lat' and 'lon' values
joined_mismatch_ok.loc[
    joined_mismatch_ok["sh_lat"].notna(), "lat"
] = joined_mismatch_ok["sh_lat"]
joined_mismatch_ok.loc[
    joined_mismatch_ok["sh_lon"].notna(), "lon"
] = joined_mismatch_ok["sh_lon"]

joined_mismatch_ok.set_index("original_index", inplace=True)

# occ_wells[occ_wells["api"].isin(mismatch_geo_ok["ok_api"])][
#     ["api", "well_name", "well_num", "operator", "sh_lat", "sh_lon", "county"]
# ]

In [ ]:
# Update 'lat' and 'lon' in the original DataFrame
registry_gdf.loc[joined_mismatch_ok.index, "lat"] = joined_mismatch_ok["lat"]
registry_gdf.loc[joined_mismatch_ok.index, "lon"] = joined_mismatch_ok["lon"]
# registry_gdf


# Update 'geometry' in the original DataFrame
registry_gdf.loc[joined_mismatch_ok.index, "geometry"] = [
    Point(xy) for xy in zip(joined_mismatch_ok.lon, joined_mismatch_ok.lat)
]
# Check which other wells may have a state value which did not agree with the spatial join result again
registry_county_gdf = registry_gdf.sjoin(
    county_fips_gdf, how="left", predicate="intersects"
).drop(columns=["index_right"])

# mismatch_geo = registry_county_gdf[registry_county_gdf["state_right"].isna()]

mismatch_geo = registry_county_gdf[
    registry_county_gdf["state_left"] != registry_county_gdf["state_right"]
]
print(f"Number of rows with mismatched state values: {mismatch_geo.shape[0]}")
# mismatch_geo

In [ ]:
# query for the wells in Texas
mismatch_geo_tx = mismatch_geo[trimmed_column_set].query(
    'state_left.str.contains("Texas")'
)
# store the original index in a new column
mismatch_geo_tx["original_index"] = mismatch_geo_tx.index

mismatch_geo_tx["tx_api"] = mismatch_geo_tx["api"].str[2:10]
# mismatch_geo_tx

#### Texas Railroad Commission (RRC)


Land survey data, bottom well data, and surface well data were taken from the RRC website. The data was then uploaded to GCP for easier reliabiliity. They are 254 zipfiles, one for each county, in the state of Texas. Each of those zipfiles contained various file extensions, and spatial data format that is usually contained in shapefiles, and contained info for various categories ranging from Airport lines to Offshore survey polys.  

In [ ]:
# regex patterns to identify which shapefiles to extract
# we are grabbing the survey lines, surface wells, and bottom well locations shp files
patterns = [r"surv\d{3}p", r"well\d{3}s", r"well\d{3}b"]

# Look at the survey lines polygons and the surface wells points. Data saved from RRC website
shp_dict = extract_matching_shp_files_from_zip_urls_concurrent(SHP_ZIP_URLS, patterns)

In [ ]:
import sys


def deep_getsizeof(obj):
    """Recursively find size of object and its elements"""
    size = sys.getsizeof(obj)
    if isinstance(obj, dict):
        size += sum(deep_getsizeof(k) + deep_getsizeof(v) for k, v in obj.items())
    elif isinstance(obj, (list, tuple)):
        size += sum(deep_getsizeof(x) for x in obj)
    return size


# Assume 'my_dict' is your dictionary
total_size_in_bytes = deep_getsizeof(shp_dict)
total_size_in_mb = total_size_in_bytes / 1e6
print(f"The total size of the dictionary is {total_size_in_mb:.2f} MB.")

In [ ]:
format_in_000(total_size_in_bytes)

In [ ]:
# use the patterns to separate the gdf in the dict based on the pattern
surv_dict = {k: shp_dict[k] for k, v in shp_dict.items() if re.search(patterns[0], k)}
swell_dict = {k: shp_dict[k] for k, v in shp_dict.items() if re.search(patterns[1], k)}
bwell_dict = {k: shp_dict[k] for k, v in shp_dict.items() if re.search(patterns[2], k)}

#### Texas RRC land survey data

In [ ]:
# O&G well symnum is a number that indicates the type of well simplified for fewer bins
well_symnum_dict = {
    2: "Permitted Location",
    3: "Dry Hole",
    4: "Oil/Gas",  # oil
    5: "Oil/Gas",  # gas
    6: "Oil/Gas",  # oil/ gas
    7: "Plugged/Shut-in",  # oil
    8: "Plugged/Shut-in",  # gas
    9: "Canceled Location",
    10: "Plugged/Shut-in",
    11: "Injection/Disposal",
    12: "Core Test",
    17: "Storage",  # oil
    18: "Storage",  # gas
    19: "Plugged/Shut-in",  # oil
    20: "Plugged/Shut-in",  # gas
    21: "Injection/Disposal",  # oil
    22: "Injection/Disposal",  # gas
    23: "Injection/Disposal",  # oil/ gas
    73: "Brine Mining",
    74: "Water Supply",
    75: "Water Supply",  # oil
    76: "Water Supply",  # gas
    77: "Water Supply",  # oil/ gas
    86: "Horizontal",  # Horizontal Well Surface Location",
    87: "Horizontal",  # Directional/Sidetrack Well Surface Location,
    88: "Storage",
    103: "Storage",  # oil/gas
}
# well_symnum_dict

In [ ]:
# Concatenate the GeoDataFrames in surv_dict into a single GeoDataFrame
surv_data_gdf = concat_gdf_from_dict(surv_dict)

# Convert the column names to snake case for consistency
surv_data_gdf.columns = [pascal_to_snake(col) for col in surv_data_gdf.columns]
# addd a coulmn for the county_code
surv_data_gdf["county_code"] = surv_data_gdf["source_file"].str.extract(r"(\d{3})")

# Display a sample of 3 rows from the DataFrame
display(surv_data_gdf.sample(3))

# Display information about the DataFrame, including the number of non-null entries in each column
surv_data_gdf.info(memory_usage="deep")

#### Texas RRC surface well data

In [ ]:
# Concatenate the GeoDataFrames in well_dict into a single GeoDataFrame
swell_data_gdf = concat_gdf_from_dict(swell_dict)

# Convert the column names to snake case for consistency
swell_data_gdf.columns = [pascal_to_snake(col) for col in swell_data_gdf.columns]
# get the county code from the source_file column
swell_data_gdf["county_code"] = swell_data_gdf["source_file"].str.extract(r"(\d{3})")
# map the dictionary to the SYMNUM column and fill the rare values with 'Other'
swell_data_gdf["well_type"] = (
    swell_data_gdf["symnum"].map(well_symnum_dict).fillna("Other")
)

# Display a sample of 3 rows from the DataFrame
display(swell_data_gdf.sample(3))

# Display information about the DataFrame, including the number of non-null entries in each column
swell_data_gdf.info(
    memory_usage="deep"
)  # shp_dict = extract_specific_gdf_from_zip_url(shp_zip_urls, patterns)

In [ ]:
swell_data_gdf["well_type"].value_counts()

In [ ]:
# plot the well types on a horizontal bar chart
swell_data_gdf["well_type"].value_counts().hvplot.barh(
    height=400,
    width=600,
    title="Surface Well Types Count",
    xlabel="",
    ylabel="",
    xaxis="bare",
    hover_cols="all",
).opts(
    active_tools=["box_zoom"],
    toolbar="above",
)

In [ ]:
# plot on a map the dry holes
dry_hole_gdf = swell_data_gdf[swell_data_gdf["well_type"] == "Dry Hole"]

# vertorize the coordinates
dry_hole_mer = platecaree_to_mercator_vectorised(
    dry_hole_gdf["geometry"].x, dry_hole_gdf["geometry"].y
)
dry_hole_coords = pd.DataFrame(dry_hole_mer, columns=["x", "y"])

bg_map = get_background_map()
# plot the dry holes on a map
bg_map * gv.Points(
    dry_hole_coords.reset_index(), ["x", "y"], ["index"], crs=ccrs.GOOGLE_MERCATOR
).opts(width=800, height=600, title="Dry Hole Locations", size=1)

In [ ]:
# create function to plot the well types on a map
def plot_well_types(gdf, well_type):
    """Plots the well types on a map"""
    # filter the swell_data_gdf for the well_type
    well_type_gdf = gdf[gdf["well_type"] == well_type]
    # vertorize the coordinates
    well_type_mer = platecaree_to_mercator_vectorised(
        well_type_gdf["geometry"].x, well_type_gdf["geometry"].y
    )
    well_type_coords = pd.DataFrame(well_type_mer, columns=["x", "y"])
    # plot the well types on a map
    return get_background_map() * gv.Points(
        well_type_coords.reset_index(),
        ["x", "y"],
        ["index"],
        crs=ccrs.GOOGLE_MERCATOR,
    ).opts(width=800, height=600, title=f"{well_type} Locations", size=1).opts(
        active_tools=["box_zoom"],
        toolbar="above",
        xaxis="bare",
        yaxis="bare",
        tools=["hover"],
    )


from functools import partial

# create a partial function for the plot_well_types function
plot_well_types_partial = partial(plot_well_types, swell_data_gdf)

plot_well_types_partial("Plugged/Shut-in")

#### Texas RRC bottom hole data

In [ ]:
# Concatenate the GeoDataFrames in well_dict into a single GeoDataFrame
bwell_data_gdf = concat_gdf_from_dict(bwell_dict)

# Convert the column names to snake case for consistency
bwell_data_gdf.columns = [pascal_to_snake(col) for col in bwell_data_gdf.columns]
# get the county code from the source_file column
bwell_data_gdf["county_code"] = bwell_data_gdf["source_file"].str.extract(r"(\d{3})")

# map the dictionary to the SYMNUM column and fill the rare values with 'Other'
bwell_data_gdf["well_type"] = (
    bwell_data_gdf["symnum"].map(well_symnum_dict).fillna("Other")
)
# Display a sample of 3 rows from the DataFrame
display(bwell_data_gdf.sample(3))

# Display information about the DataFrame, including the number of non-null entries in each column
bwell_data_gdf.info(memory_usage="deep")

#### matching the mismatched wells

In [ ]:
swell_columns = ["api", "lat83", "long83", "well_type"]
mismatch_columns = ["tx_api", "lon", "lat", "original_index"]

In [ ]:
mismatch_geo_tx[mismatch_columns].merge(
    swell_data_gdf[swell_columns], how="left", left_on="tx_api", right_on="api"
)

In [ ]:
joined_mismatch_tx = mismatch_geo_tx[mismatch_columns].merge(
    swell_data_gdf[swell_columns], how="left", left_on="tx_api", right_on="api"
)

# set the index to the original_index column
joined_mismatch_tx.set_index("original_index", inplace=True)

# replace the lat and lon values with the lat83 and lon83 values if not nan
joined_mismatch_tx.loc[joined_mismatch_tx["lat83"].notna(), "lat"] = joined_mismatch_tx[
    "lat83"
]
joined_mismatch_tx.loc[
    joined_mismatch_tx["long83"].notna(), "lon"
] = joined_mismatch_tx["long83"]

# Update 'lat' and 'lon' in the original DataFrame


registry_gdf.loc[joined_mismatch_tx.index, "lat"] = joined_mismatch_tx["lat"]
registry_gdf.loc[joined_mismatch_tx.index, "lon"] = joined_mismatch_tx["lon"]


# Update 'geometry' in the original DataFrame
registry_gdf.loc[joined_mismatch_tx.index, "geometry"] = [
    Point(xy) for xy in zip(joined_mismatch_tx.lon, joined_mismatch_tx.lat)
]

In [ ]:
registry_gdf.shape

In [ ]:
# Check which other wells may have a state value which did not agree with the spatial join result again
registry_county_gdf = registry_gdf.sjoin(
    county_fips_gdf, how="left", predicate="intersects"
).drop(columns=["index_right"])
mismatch_geo = registry_county_gdf[registry_county_gdf["state_right"].isna()]
mismatch_geo.shape
print(f"Number of rows with mismatched state values: {mismatch_geo.shape[0]}")
display(mismatch_geo.sample(3))

# drop the rows with null values in the state_right column of mismatch_geo from the registry_gdf
print(f"Number of rows before dropping: {registry_gdf.shape[0]}")
registry_gdf.drop(mismatch_geo.index, inplace=True)

In [ ]:
registry_gdf.sjoin(county_fips_gdf, how="left", predicate="intersects").drop(
    columns=["index_right"]
)[trimmed_column_set]

In [ ]:
# list the columns to keep
columns_we_want = [
    "api",
    "well",
    "state_code",
    "state_left",
    "state_right",
    "county_code",
    "county_left",
    "countyfp",
    "name",
    "operator",
    "lat",
    "lon",
    "geoid",
    "geometry",
]
registry_county_gdf = registry_gdf.sjoin(
    county_fips_gdf, how="left", predicate="intersects"
).drop(columns=["index_right"])[columns_we_want]

print(f"Number of rows: {registry_county_gdf.shape[0]}")
registry_county_gdf[
    registry_county_gdf["state_left"] != registry_county_gdf["state_right"]
][columns_we_want].sample(3)

In [ ]:
# get the rows where the state_left is California with the state_right being Texas
# This would represent where the geometry said Texas but the api or original dataset had California
# we check the api with
cali_tx_query = registry_county_gdf.query(
    'state_left == "California" & state_right == "Texas"'
)
# from the query, take the last 9 of the api (well_id) with 42 at beginning
# with county_code from county that was matched with the geometry
cali_tx_query["state_code"] = "42"
cali_tx_query["county_code"] = cali_tx_query["countyfp"]
cali_tx_query["well_id"] = cali_tx_query["api"].str[-9:]
cali_tx_query["api"] = (
    cali_tx_query["state_code"]
    + cali_tx_query["county_code"]
    + cali_tx_query["well_id"]
)

# put api, county_code and state_code into the registry_gdf
registry_gdf.loc[
    cali_tx_query.index, ["api", "county_code", "state_code"]
] = cali_tx_query[["api", "county_code", "state_code"]].values

In [ ]:
registry_county_gdf[
    registry_county_gdf[columns_we_want]["state_left"]
    != registry_county_gdf[columns_we_want]["state_right"]
][columns_we_want]

In [ ]:
# swell_data_gdf["well_type"].value_counts()
# well_data_gdf[well_data_gdf["well_type"] == "Other"]["symnum"].value_counts()
# look for the mismatch_geo_tx api in the swell_data_gdf and get those rrows

mismatch_geo_tx_sw = swell_data_gdf[
    swell_data_gdf["api"].isin(mismatch_geo_tx["tx_api"])
].copy()
mismatch_geo_tx_sw
# convert the long83 and lat83 columns to geomety Points
mismatch_geo_tx_sw["geometry"] = [
    Point(xy) for xy in zip(mismatch_geo_tx_sw.long83, mismatch_geo_tx_sw.lat83)
]
# create a GeoDataFrame from the mismatch_geo_tx_sw dataframe
mismatch_geo_tx_sw = gpd.GeoDataFrame(
    mismatch_geo_tx_sw, geometry="geometry", crs="EPSG:4269"
)
# plot the mismatch_geo_tx_sw GeoDataFrame
# bg_map * gv.Points(mismatch_geo_tx_sw).opts(height=500, width=800, tools=["hover"])

mismatch_geo_tx_sw.sjoin(county_fips_gdf, how="left", predicate="intersects")

In [ ]:
# check the length of the tx_api values
swell_data_gdf["api"].astype("string").str.len().value_counts()

# drop the rows with the api number length of 3
swell_data_gdf.drop(
    swell_data_gdf[swell_data_gdf["api"].astype("string").str.len() == 3].index,
    inplace=True,
)

In [ ]:
test_zip = SHP_ZIP_URLS[1]

In [ ]:
# get the zip file at test_zip
with urlopen(test_zip) as u:
    zip_data = u.read()
# extract the zip file
with ZipMemoryFile(zip_data) as z:
    zip_files = z.listdir()
    print(zip_files)
    # get the .shp files
    shp_files = [f for f in zip_files if f.endswith(".shp")]
    # read in the .shp files
    for shp_file in shp_files:
        with z.open(shp_file) as f:
            gdf = gpd.GeoDataFrame.from_features(f, crs=f.crs)
            # add column for the source file
            gdf["source_file"] = shp_file
            print(f"Shape of {shp_file}: {gdf.shape}")
            display(gdf.sample())

In [ ]:
bg_map = get_background_map()

ok_counties = county_fips_gdf.query('state.str.contains("Oklahoma")').copy()
ok_counties.to_crs("EPSG:3857", inplace=True)

tx_counties = county_fips_gdf.query('state.str.contains("Texas")').copy()
tx_counties.to_crs("EPSG:3857", inplace=True)

ok_polys = gv.Polygons(ok_counties, crs=ccrs.GOOGLE_MERCATOR).opts(
    projection=ccrs.GOOGLE_MERCATOR,
    line_color="black",
    fill_alpha=0,
    height=500,
    width=500,
)
bg_map * ok_polys

In [ ]:
# Define a list of states that are in the Permian Basin.
nm_tx = ["New Mexico", "Texas"]

# Filter the county_fips_gdf DataFrame to include only the counties in the Permian states.
counties_nm_tx_gdf = county_fips_gdf[county_fips_gdf["state"].isin(nm_tx)]

# Perform a spatial join between the registry_gdf and counties_nm_tx_gdf DataFrames.
# This will add the data from counties_nm_tx_gdf to registry_gdf for matching locations.
# After the join, drop the 'index_right' column as it's not needed.
registry_nm_tx_gdf = registry_gdf.sjoin(counties_nm_tx_gdf).drop(
    columns=["index_right"]
)
registry_nm_tx_gdf.sample(3)

In [ ]:
registry_nm_tx_gdf.info()

In [ ]:
registry_nm_tx_gdf["operator_name"].nunique()

In [ ]:
# create a pivot table with the count of operators for each year
operator_year_count = registry_nm_tx_gdf.pivot_table(
    index="operator_name", columns="job_start_year", values="api", aggfunc="count"
)
# See which operators were active every year
# operator_year_count[operator_year_count.count(axis=1) == 11]

# See who has been active for the last 5 years
operator_active_5y = operator_year_count.loc[
    ~operator_year_count.iloc[:, -5:].isna().any(axis=1)
].fillna(0)

# do some styler table formatting from pandas
style = operator_active_5y.style.background_gradient(
    cmap="cet_CET_L2_r", axis=1, vmin=0, vmax=operator_year_count.max().max()
)
# Format the numbers in the table as integers
style = style.format("{:.0f}")
print(f"Number of operators active for the last 5 years: {len(operator_active_5y)}")
style

In [ ]:
# Create a mask for rows where the first 2 characters of the api number are not 42 nor 30.
# This is done to filter out rows that do not belong to the states we are interested in (Texas and New Mexico).
api_mask = ~registry_nm_tx_gdf["api"].str[0:2].isin(["42", "30"])

# Apply the mask to the registry_nm_tx_gdf DataFrame to get the rows that match the condition.
mismatch_state = registry_nm_tx_gdf[api_mask]

# Display selected columns from the mismatch_state DataFrame.
# These columns provide information about the well, its location, and the job start date.
display(
    mismatch_state[
        [
            "api",
            "api_number",
            "state_right",
            "state_name",
            "state_number",
            "well_name",
            "operator_name",
            "county_name",
            "latitude",
            "longitude",
            "geometry",
            "crs",
            "job_start_date",
            "county",
            "countyfp",
        ]
    ]
)

In [ ]:
# get a background map
bg_map = get_background_map()

In [ ]:
# Convert the coordinates to Mercator
mercator_coords = platecaree_to_mercator_vectorised(
    registry_nm_tx_gdf["geometry"].x, registry_nm_tx_gdf["geometry"].y
)

# Round the coordinates and create a DataFrame
mer_points = pd.DataFrame(np.round(mercator_coords), columns=["x", "y"])

# Create a Points object for plotting
gpoints = gv.Points(
    mer_points.reset_index(), ["x", "y"], ["index"], crs=ccrs.GOOGLE_MERCATOR
).opts(height=600, width=800, color="skyblue", size=1, tools=["hover"])

# Create a layout with the background map and the points
layout = bg_map * gpoints
layout

### Map files taken from the EIA website.


### Formations

In [ ]:
# Use the function 'extract_gdfs_from_zip_url_concurrent' to get GeoDataFrames from the URLs in 'basins_url_list'
# This function concurrently downloads and extracts GeoDataFrames from the given URLs
basins_dict = extract_gdfs_from_zip_url_concurrent(basins_url_list)

# Display the keys of 'basins_dict' to see the names of the basins
display(basins_dict.keys())

# Concatenate the GeoDataFrames in 'basins_dict' into a single GeoDataFrame using the function 'concat_gdf_from_dict'
basins_gdf = concat_gdf_from_dict(basins_dict)

# Convert the column names of 'basins_gdf' to snake case for consistency
# The function 'pascal_to_snake' is used to convert PascalCase or camelCase to snake_case
basins_gdf.columns = [pascal_to_snake(col) for col in basins_gdf.columns]

In [ ]:
print(f"Number of sub basins/ geodataframes: {len(basins_dict)}\n")

for k, gdf in basins_dict.items():
    print(f"{k}| Shape:{gdf.shape}| CRS:{gdf.crs.to_string()}")
    display(gdf.sample())
    print()

In [ ]:
# get the shapefile of the basin boundaries
basins_dict = extract_gdfs_from_zip_url_concurrent(basins_url_list)
display(basins_dict.keys())
basins_gdf = concat_gdf_from_dict(basins_dict)
# scrub the column names
basins_gdf.columns = [pascal_to_snake(col) for col in basins_gdf.columns]

In [ ]:
# Plot shale plays and basin boundaries of the different formations in the Permian Basin
bg_map * basins_gdf.hvplot(
    geo=True,
    alpha=0.5,
    title="Shale Plays in the Permian Basin",
    legend=True,
    by="shale_play",
    muted_alpha=0.01,
).opts(
    tools=["hover", "tap"],
    legend_position="right",
    height=600,
    width=800,
)

In [ ]:
from holoviews import opts

# Dissolve the geometries of the basins_gdf GeoDataFrame into a single geometry


shale_plays_gdf = basins_gdf[["geometry"]].dissolve()
# get the counties in the Permian Basin
permian_counties = county_fips_gdf.intersects(shale_plays_gdf)
# plot the counties in the Permian Basin
bg_map * gv.Polygons(county_fips_gdf[permian_counties]).opts(
    title="Counties in the Permian Basin",
    tools=["hover"],
    height=600,
    width=800,
    alpha=0.5,
    color="skyblue",
)
# plot an outline of the Permian Basin over the counties
overlay = (
    bg_map
    * gv.Polygons(county_fips_gdf[permian_counties])
    * gv.Path(shale_plays_gdf)
    # * gpoints
)

overlay.opts(
    opts.Polygons(alpha=0.5, cmap=["#73d2ff"], line_color="gray"),
    opts.Path(alpha=0.5, color="black"),
    opts.Overlay(tools=["hover"], height=600, width=800),
    opts.Points(color="crimson"),
)


# plot to confirm that the geometries have been dissolved
# bg_map * gv.Polygons(shale_plays_gdf).opts(
#     # geo=True,
#     title="Dissolved Shale play in the Permian Basin",
#     height=600,
#     width=800,
# )

#### State land leases

#### New Mexico:
> 


In [ ]:
# Use the extract_gdfs_from_zip_url_concurrent function to download and extract GeoDataFrames
# from the shapefile zip files at the URLs in the shp_url_list. The function returns a dictionary
# where the keys are the names of the shapefiles and the values are the corresponding GeoDataFrames.
nm_slo_dict = extract_gdfs_from_zip_url_concurrent(nm_slo_url_list)

# Display the keys of the land_map_dict dictionary. These are the names of the shapefiles
# that were downloaded and extracted.
nm_slo_dict.keys()

In [ ]:
# land_map_dict = extract_gdfs_from_zip_url_concurrent(shp_url_list)


# land_map_dict.keys()

In [ ]:
# sample the gdfs in the dictionary
for k, gdf in nm_slo_dict.items():
    print(f"{k}| Shape:{gdf.shape}| CRS:{gdf.crs.to_string()}")
    display(gdf.sample(3))
    display(gdf.info())

In [ ]:
# Create 2 separate gdfs instead of concatenating them as they have distinct columns
nm_slo_gdfs = list(nm_slo_dict.values())
# first one is the geologic regions
nm_slo_geo = nm_slo_gdfs[0]
# scrub the columns
nm_slo_geo.columns = [pascal_to_snake(col) for col in nm_slo_geo.columns]

# Define  a dictionary for the opts to include in plot function
poly_opts = dict(
    alpha=0.8,
    height=600,
    width=800,
    line_width=0,
    line_color="lightgray",
    tools=["hover"],
)


# Adjust opts for this plot
poly_opts_copy = poly_opts.copy()
poly_opts_copy["line_width"] = 1

# plot the geologic regions gdf
bg_map * gv.Polygons(nm_slo_geo.to_crs("EPSG:4269"), vdims=["label"]).opts(
    **poly_opts_copy, cmap=["#73d2ff"] * 256, title="New Mexico Geologic Regions"
)

In [ ]:
# Second one is the oil and gas leases on New Mexico State Trust Lands
nm_slo_lease = nm_slo_gdfs[1]
# scrub the columns
nm_slo_lease.columns = [pascal_to_snake(col) for col in nm_slo_lease.columns]
# create a new column for the area of the lease
nm_slo_lease["area"] = nm_slo_lease["geometry"].area
# groupby the ogrid_nam and sum the area
# add the transformed area to the gdf
nm_slo_lease["ogrid_area"] = (
    nm_slo_lease.groupby("ogrid_nam")["area"].transform("sum") / 1e6
)

# plot the oil and gas leases gdf for New Mexico State Trust Lands
bg_map * gv.Polygons(
    nm_slo_lease.to_crs("EPSG:4269"), vdims=["ogrid_nam", "ogrid_area"]
).opts(
    **poly_opts,
    cnorm="eq_hist",
    colorbar=True,
    title="Oil and Gas Leases on New Mexico State Trust Lands"
)

In [ ]:
# list(land_map_dict.values())
# [gdf['geometry'] for gdf in land_map_dict.values()]
random_color_list = [
    "#" + "".join([random.choice("0123456789ABCDEF") for j in range(6)])
    for i in range(len(nm_slo_dict))
]
plots = []
new_map = get_background_map()
plots.append(new_map)
for color, (name, gdf) in zip(random_color_list, nm_slo_dict.items()):
    # Add new column with the name of the shapefile for the hover tool
    gdf["label"] = name
    plot = gv.Polygons(gdf.to_crs("EPSG:4269"), vdims=["label"]).opts(
        tools=["hover"], height=600, width=800, alpha=0.5, title=""
    )
    plots.append(plot)

overlay = hv.Overlay(plots)
overlay

In [ ]:
nm_slo_lease.info()

In [ ]:
# Concatenate the GeoDataFrames in well_dict into a single GeoDataFrame
swell_data_gdf = concat_gdf_from_dict(swell_dict)

# Convert the column names to snake case for consistency
swell_data_gdf.columns = [pascal_to_snake(col) for col in swell_data_gdf.columns]
# get the county code from the source_file column
swell_data_gdf["county_code"] = swell_data_gdf["source_file"].str.extract(r"(\d{3})")

# Display a sample of 3 rows from the DataFrame
display(swell_data_gdf.sample(3))

# Display information about the DataFrame, including the number of non-null entries in each column
swell_data_gdf.info()

In [ ]:
# add a column to the surv_data_gdf with the county number
# the county_number wil be the numbers in the source_file column
surv_data_gdf["county_number"] = surv_data_gdf["source_file"].str.extract(r"(\d{3})")

# using just the geometry and the county_number columns, intersect with the permian basin gdf
surv_permian_gdf = surv_data_gdf[["geometry", "county_number"]].sjoin(
    shale_plays_gdf[["geometry"]], how="inner", predicate="intersects"
)
# see which counties are in the permian basin
pb_county_numbers = surv_permian_gdf["county_number"].unique().tolist()

# plot the survey lines in the permian basin
bg_map * gv.Polygons(
    surv_permian_gdf.to_crs("EPSG:4269"), vdims=["county_number"]
).opts(
    tools=["hover"],
    height=600,
    width=800,
    alpha=0.5,
    line_width=0,
    title="Permian Basin Survey Lines",
)

# surv_data_gdf.sample(3)

In [ ]:
# see how land survey polygon data looks on map
pb_plot = shale_plays_gdf.hvplot(geo=True, color="red", alpha=0.5, line_width=0).opts(
    height=600, width=800
)
survey_plot = surv_data_gdf.hvplot(geo=True, color="blue", alpha=0.5, line_width=0)

# bg_map * survey_plot * pb_plot

In [ ]:
# Get the intersection of the survey polygons and the Permian Basin polygon
survey_pb_gdf = gpd.overlay(surv_data_gdf, shale_plays_gdf, how="intersection")
survey_pb_gdf.info()

In [ ]:
# survey_pb_gdf.explore()
# surv_data_gdf.sample(100)

In [ ]:
# spatial join of the registry_gdf(from fracfocus) and surv_data_gdf
registry_join_gdf = gpd.sjoin(
    registry_gdf[
        [
            "geometry",
            "api",
            "operator_name",
            "well_name",
            "state",
            "county_name",
            "county_number",
        ]
    ],
    surv_data_gdf,
).drop(columns=["index_right"])


registry_join_gdf.sort_values(by="api")

registry_join_gdf.county_name.value_counts()

# create a well_id column from the api column

registry_join_gdf["tx_api"] = registry_join_gdf["api"].str[2:10]


# merge the welltype column from well_data_gdf to registry_join_gdf on the api_short column

registry_join_gdf = (
    registry_join_gdf.merge(swell_data_gdf[["tx_api", "well_type"]], on="tx_api")
    .drop(columns=["scrap_file", "level4_sur"])
    .rename(columns={"level2_blo": "block"})
)
# registry_join_gdf.explore()
# plot polygons using geoviews
bg_map * gv.Polygons(registry_join_gdf.to_crs("EPSG:4269"), vdims=["well_type"]).opts(
    **poly_opts, color="well_type", title="Well Types in the Permian Basin"
)

bg_map * registry_join_gdf.hvplot(
    geo=True,
    by="well_type",
    alpha=0.8,
    legend="right",
    width=800,
    height=600,
    size=1,
    muted_alpha=0.1,
)

### Geodatabase files taken from the Texas GLO (General Land Office.)

These files contained both the Oil and Gas Leases (active only), managed by the Texas GLO, and Oil & Gas units (active only) which is Oil and Gas pooling agreements managed by the Texas GLO. 

In [ ]:
# get the geodataframe of the active leases
# active_gdb_dict = read_gdb_from_zip_url(gdb_zip_urls)


# get the geodataframe of the active leases using concurrent futures
active_gdb_dict = read_gdb_from_zip_url_concurrent(GDB_ZIP_URLS)

In [ ]:
active_gdb_dict

In [ ]:
active_gdb_dict.keys()

In [ ]:
# Read in the active lease geodatabase
active_leases_gdf = active_gdb_dict["OAG_Leases_Active"]
# clean column names
active_leases_gdf.columns = [pascal_to_snake(col) for col in active_leases_gdf.columns]
active_leases_gdf.columns

In [ ]:
# get the columns with the date in it using regex
date_cols = [col for col in active_leases_gdf.columns if re.search(r"date", col)]
# add any other columns that should be dates
date_cols.extend(["lease_input"])

date_cols

In [ ]:
# convert the date columns to datetime
active_leases_gdf[date_cols] = pd.concat(
    [pd.to_datetime(active_leases_gdf[col]) for col in date_cols], axis=1
)
# active_leases_gdf[date_cols] = active_leases_gdf[date_cols].fillna(
#     pd.Timestamp("1900-06-28")
# )

In [ ]:
# get the columns interested in seeing
columns_of_interest = date_cols + [
    "county",
    "geometry",
    "land_type",
    "primary_term_year",
    "original_lessee",
    "lessor",
    "field_name",
    "lease_type",
    "lease_status",
]

In [ ]:
active_leases_gdf[columns_of_interest].info()
active_leases_gdf[active_leases_gdf[columns_of_interest].isna().any(axis=1)][
    columns_of_interest
].sort_values(by="effective_date", ascending=False)

In [ ]:
non_date_cols = list(set(columns_of_interest) - set(date_cols))
pd.concat(
    [
        active_leases_gdf[non_date_cols],
        active_leases_gdf[date_cols].astype(
            str
        ),  # the .explore() does not work with NaT in datetime columns
    ],
    axis=1,
).explore()

### Dask

 #### Production Data Query Dump from RRC

In [ ]:
# Create a Dask client with specified configuration
# This client will use 4 workers, each with 1 thread and a memory limit of 2GB
# The Dask diagnostic dashboard will be served at the address ":8788"
client = Client(
    n_workers=4, threads_per_worker=1, memory_limit="2GB", dashboard_address=":8788"
)

# Print the link to the Dask diagnostic dashboard
# This link can be used to monitor the progress and performance of Dask computations
print(client.dashboard_link)

In [ ]:
# general purpose table paths
county_path = Path("../data/PDQ_DSV/GP_COUNTY_DATA_TABLE.dsv")
district_path = Path("../data/PDQ_DSV/GP_DISTRICT_DATA_TABLE.dsv")
date_range_path = Path("../data/PDQ_DSV/GP_DATE_RANGE_CYCLE_DATA_TABLE.dsv")

# production report data paths
county_cycle_path = Path("../data/PDQ_DSV/OG_COUNTY_CYCLE_DATA_TABLE.dsv")
county_lease_path = Path("../data/PDQ_DSV/OG_COUNTY_LEASE_CYCLE_DATA_TABLE.dsv")
district_cycle_path = Path("../data/PDQ_DSV/OG_DISTRICT_CYCLE_DATA_TABLE.dsv")

field_cycle_path = Path("../data/PDQ_DSV/OG_FIELD_CYCLE_DATA_TABLE.dsv")
field_info_path = Path("../data/PDQ_DSV/OG_FIELD_DW_DATA_TABLE.dsv")

lease_cycle_path = Path("../data/PDQ_DSV/OG_LEASE_CYCLE_DATA_TABLE.dsv")
lease_disposition_path = Path("../data/PDQ_DSV/OG_LEASE_CYCLE_DISP_DATA_TABLE.dsv")
lease_info_path = Path("../data/PDQ_DSV/OG_REGULATORY_LEASE_DW_DATA_TABLE.dsv")

operator_cycle_path = Path("../data/PDQ_DSV/OG_OPERATOR_CYCLE_DATA_TABLE.dsv")
operator_info_path = Path("../data/PDQ_DSV/OG_OPERATOR_DW_DATA_TABLE.dsv")
# summary tables
summary_master_path = Path("../data/PDQ_DSV/OG_SUMMARY_MASTER_LARGE_DATA_TABLE.dsv")
summary_onshore_path = Path("../data/PDQ_DSV/OG_SUMMARY_ONSHORE_LEASE_DATA_TABLE.dsv")
# well completion data
well_comp_path = Path("../data/PDQ_DSV/OG_WELL_COMPLETION_DATA_TABLE.dsv")

pdq_dsv_paths = [
    county_path,
    district_path,
    date_range_path,
    county_cycle_path,
    county_lease_path,
    district_cycle_path,
    field_cycle_path,
    field_info_path,
    lease_cycle_path,
    lease_disposition_path,
    lease_info_path,
    operator_cycle_path,
    operator_info_path,
    summary_master_path,
    summary_onshore_path,
    well_comp_path,
]

In [ ]:
def get_table_overview(path):
    """
    Returns an overview of the data in the file at the given path.

    This function returns a dictionary containing the file name, the number of rows,
    the column names, the first five rows of the data, and the column data types.
    It uses Dask to efficiently compute the number of rows in the file, which makes
    it suitable for large files.

    Parameters:
    path (Path or str): The path to the data file.

    Returns:
    dict: A pandas DataFrame containing the file overview.
    """
    column_types = {}
    # Initialize ddf as an empty Dask DataFrame
    while True:
        try:
            ddf = dd.read_csv(path, sep="}", dtype=column_types)
            num_rows = len(ddf)
            break
        except ValueError as e:
            match = re.search(r"dtype=\{'(.*?)': '(.*?)'", str(e))
            # match = re.search(r"dtype=\{'(.*?)': 'object'", str(e))
            if match:
                column_name = match.group(1)
                column_types[column_name] = "object"

    sample_data = pd.read_csv(path, sep="}", nrows=1000, dtype=column_types)
    columns = sample_data.columns.tolist()

    # column_dtypes = ddf.dtypes.tolist()
    column_dtypes = sample_data.dtypes.tolist()

    overview = pd.DataFrame(
        {
            "file_name": [path.stem],
            "file_extension": [path.suffix],
            "num_rows": [num_rows],
            "num_cols": [len(columns)],
            "cols": [columns],
            "col_dtypes": [column_dtypes],
            # "sample_data": [sample_data.to_json()],
        },
    )
    return overview

In [ ]:
def display_table_overview(path):
    """
    Displays an overview of the data in the file at the given path.

    This function prints the file name, the number of rows, the column names,
    and the first five rows of the data. It uses Dask to efficiently compute
    the number of rows in the file, which makes it suitable for large files.

    Parameters:
    path (Path or str): The path to the data file.
    """
    # make a decorative line
    decor_length = len(path.name) + 8
    decor = "~*~" * (decor_length // 3)

    print(f"{decor}")
    print(f"File: {path.name}")
    print(f"{decor}")

    column_types = {}

    while True:
        try:
            ddf = dd.read_csv(path, sep="}", dtype=column_types)
            print(f"Number of rows: {len(ddf)}")
            break
        except ValueError as e:
            # print(e)
            # If a ValueError occurs, parse the error message to get the column name
            match = re.search(r"dtype=\{'(.*?)': '(.*?)'", str(e))
            # match = re.search(r"dtype=\{'(.*?)': 'object'", str(e))
            if match:
                column_name = match.group(1)
                print(f"Problematic column: {column_name}")
                # Add the problematic column to the column types dictionary
                column_types[column_name] = "object"

    # columns = pd.read_csv(
    #     path, delimiter="}", nrows=1, dtype=column_types
    # ).columns.tolist()
    columns = ddf.columns.tolist()

    print(f"Number of columns: {len(columns)}")
    print(f"Columns: {columns}")
    print(f"Column data types: {ddf.dtypes.tolist()}")

    print(f"Sample of data in {path.name}:")
    display(pd.read_csv(path, delimiter="}", nrows=20, dtype=column_types))

In [ ]:
# display_table_overview(operator_cycle_path)
# pd.read_csv(operator_cycle_path, sep="}", nrows=1000).dtypes
# column_dtypes

In [ ]:
# get the table overview for each file in the pdq_dsv_paths list
tables_list = []
for fpath in pdq_dsv_paths:
    print(f"Getting overview for {fpath.stem}")
    tables_list.append(get_table_overview(fpath))

# pdq_dsv_overviews = [get_table_overview(path) for path in tqdm(pdq_dsv_paths)]

In [ ]:
# concatenate the tables_list into a single DataFrame
table_samples_df = pd.concat(tables_list, ignore_index=True)
table_samples_df.iloc[:, 0:5]

In [ ]:
# func to get the column data types
def get_column_dtypes(table, row_number):
    """Returns a dictionary of the column names and their data types"""
    # get the column names from the table
    columns = table.loc[row_number, "cols"]
    # get the column data types from the table
    column_dtypes = table.loc[row_number, "col_dtypes"]
    # create a dictionary of the column names and their data types
    column_dtypes_dict = dict(zip(columns, column_dtypes))
    return column_dtypes_dict

In [ ]:
from collections import defaultdict

# row_num = 0
# create a defaultdict of dicts
table_col_dtype_dict = defaultdict(dict)
for i in range(len(table_samples_df)):
    # get the column data types for the row
    column_dtypes_dict = get_column_dtypes(table_samples_df, i)
    # add the column data types to the defaultdict
    table_col_dtype_dict[i].update(column_dtypes_dict)


In [ ]:
row_num = 0
# row 0 is the general purpose county table

county_gp_ddf = dd.read_csv(
    pdq_dsv_paths[row_num], sep="}", dtype=table_col_dtype_dict[row_num]
)
county_gp_ddf.columns = [pascal_to_snake(col) for col in county_gp_ddf.columns]
county_gp_ddf["county_code"] = (
    county_gp_ddf["county_fips_code"].astype("string").str.zfill(3)
)
county_gp_ddf = county_gp_ddf[county_gp_ddf["on_shore_flag"] == "Y"]
county_gp_ddf = county_gp_ddf.drop(
    columns=[
        "county_fips_code",
        "county_no",
        "on_shore_flag",
        "onshore_assc_cnty_flag",
    ]
)
county_gp_ddf

# get_column_dtypes(table_samples_df, row_number=row_num)


# ddf_county_gp.columns = [pascal_to_snake(col) for col in ddf_county_gp.columns]
# ddf_county_gp.head(5)

# nan_columns = ddf_county_gp.columns[ddf_county_gp.isna().all().compute()]
# ddf_county_gp = ddf_county_gp.drop(columns=nan_columns)

In [ ]:
row_num = 3
# row 3 is the county cycle table

county_cycle_ddf = dd.read_csv(
    pdq_dsv_paths[row_num], sep="}", dtype=table_col_dtype_dict[row_num]
)
county_cycle_ddf.columns = [pascal_to_snake(col) for col in county_cycle_ddf.columns]
# filter for years after 2012
county_cycle_ddf = county_cycle_ddf[county_cycle_ddf["cycle_year"] > 2012]
county_cycle_ddf["county_code"] = (
    county_cycle_ddf["county_no"].astype("string").str.zfill(3)
)
# filter for the county_code in the county_gp_ddf
county_cycle_ddf = county_cycle_ddf[county_cycle_ddf["county_code"] < "508"]
# county_cycle_ddf.head(20)

In [ ]:
# group by county_code and cyclle_year and sum cnty_oil_prod_vol
county_cycle_grouped = county_cycle_ddf.groupby(["county_code", "cycle_year"])
county_cycle_grouped = county_cycle_grouped[
    "cnty_oil_prod_vol", "cnty_gas_prod_vol", "cnty_cond_prod_vol", "cnty_csgd_prod_vol"
].sum()
county_cycle_computed = county_cycle_grouped.compute()
county_cycle_computed.reset_index()

In [ ]:
# county_cycle_computed = county_cycle_computed.reset_index()
# get only the top 3 counties by oil production for each year
max_oil_county_index = (
    county_cycle_computed.reset_index()
    .groupby("cycle_year")["cnty_oil_prod_vol"]
    .idxmax()
)
max_gas_county_index = county_cycle_computed.groupby("cycle_year")[
    "cnty_gas_prod_vol"
].idxmax()
max_cond_county_index = county_cycle_computed.groupby("cycle_year")[
    "cnty_cond_prod_vol"
].idxmax()
max_csgd_county_index = county_cycle_computed.groupby("cycle_year")[
    "cnty_csgd_prod_vol"
].idxmax()

top_oil_counties = county_cycle_computed.loc[max_oil_county_index]
top_gas_counties = county_cycle_computed.loc[max_gas_county_index]
top_cond_counties = county_cycle_computed.loc[max_cond_county_index]
top_csgd_counties = county_cycle_computed.loc[max_csgd_county_index]
top_counties = pd.concat(
    [top_oil_counties, top_gas_counties, top_cond_counties, top_csgd_counties]
).drop_duplicates()
top_counties.sort_values(by=["cycle_year"], ascending=False)

computed_county_gp = county_gp_ddf.compute()
# merge the top_counties with the county_gp_ddf
top_counties = top_counties.merge(computed_county_gp, on="county_code")
top_counties.sort_values(by=["cycle_year"], ascending=False)

In [ ]:
ddf_lease = dd.read_csv(
    lease_cycle_path,
    sep="}",
    dtype={"DISTRICT_NAME": str, "GAS_WELL_NO": str},
)
ddf_lease.columns = [pascal_to_snake(col) for col in ddf_lease.columns]

In [ ]:
ddf_lease.columns

In [ ]:
# Filter to a certain yearse
ddf_lease_filtered = ddf_lease[ddf_lease["cycle_year"] > 2012]

# repartition the dataframe
# ddf_lease_filtered = ddf_lease_filtered.repartition(npartitions=1000)

operators_by_lease = ddf_lease_filtered["operator_name"].value_counts()

top_operators_by_lease = operators_by_lease.nlargest(10)

# top_operators_by_lease = ddf_lease_filtered["operator_name"].value_counts().nlargest(10)

desc = ddf_lease_filtered.describe()

In [ ]:
top_operators_computed, operators_computed = dask.compute(
    top_operators_by_lease, operators_by_lease
)

top_operators_computed.hvplot.barh(
    height=400,
    width=600,
    title="Top 10 Operators by Lease Count",
    xlabel="",
    ylabel="",
    xaxis="bare",
    hover_cols="all",
).opts(
    active_tools=["box_zoom"],
    toolbar="above",
)

In [ ]:
operators_computed

In [ ]:
grouped = ddf_lease_filtered.groupby(["cycle_year", "operator_name", "district_no"])[
    "lease_oil_prod_vol",
    "lease_gas_prod_vol",
    "lease_cond_prod_vol",
    "lease_csgd_prod_vol",
].sum()

# grouped.visualize(optimize_graph=True, rankdir="LR")

In [ ]:
grouped_computed = grouped.compute()

In [ ]:
def get_max_product(grouped_computed, product):
    max_index = (
        grouped_computed.reset_index()
        .groupby(["cycle_year", "district_no"])[f"lease_{product}_prod_vol"]
        .idxmax()
    )
    result = grouped_computed.reset_index().loc[max_index]
    result["product"] = product
    return result



In [ ]:
# Use the function for different products
oil_result = get_max_product(grouped_computed, "oil")
gas_result = get_max_product(grouped_computed, "gas")
cond_result = get_max_product(grouped_computed, "cond")
csgd_result = get_max_product(grouped_computed, "csgd")

# Concatenate the results
final_result = pd.concat([oil_result, gas_result, cond_result, csgd_result])

In [ ]:
final_result.sort_values(by=["cycle_year", "district_no", "product"]).info(
    memory_usage="deep"
)

In [ ]:
# Create a new Dask client with a higher memory limit
client = Client(memory_limit="10GB")

In [ ]:
# Define the path to the CSV file
# lease_path = "../data/raw/lease_table.csv"

# Define the column types
column_types = {"DISTRICT_NAME": str, "GAS_WELL_NO": str}

# Read the CSV file with Dask
lease_table_df = dd.read_csv(lease_cycle_path, sep="}", blocksize=1e8, dtype=column_types)

# Convert column names to snake case
lease_table_df.columns = [pascal_to_snake(col) for col in lease_table_df.columns]

# Repartition the DataFrame into 10 partitions
lease_table_df = lease_table_df.repartition(npartitions=10)

# Save the DataFrame as a Parquet file
lease_table_df.to_parquet("../data/processed/og_lease_cycle_data_table.parquet")

In [ ]:
info = client.scheduler_info()  # get scheduler info

workers = info["workers"]  # get the workers

print(f"Number of workers: {len(workers)}")

In [ ]:
client.close()